In [1]:
#Get the mongo table where are stored the games

import pymongo
client = pymongo.MongoClient()
db = client.game

#Create Mongo table for new games
mongoTable = db["gameTT2"]

In [2]:
#Get the winning team
def getWinner(game):
    for team in game["teams"]:
        if team["win"]=="Win":
            return team["teamId"]

In [3]:
#Need to classify teams as following jungler meta or hypercarry meta
#Intuition : hypercarry meta team needs a support, which usually has a specific support item
#            A player having one of those specific items are support
#            A team having one (or more) support player follows hypercarry meta, a team which doesn't follows the jungler meta

#List of support items (as of LoL season 8, up to patch 8.18)
supportItems = [3303, 3309, 3098, 3096, 3097, 3312, 3092, 3311, 3307, 3301, 3306, 3401, 3310, 3304, 3302, 4301, 4302, 3308, 3069, 3305]

#Detect if a player has a support item, and tag it has support
def isSupport(participant):
    for i in range(0,7):
        if participant['stats']['item'+str(i)] in supportItems:
            return True
    return False

#Check if the specified team has a support
def hasSupport(game, teamId):
    for p in game['participants']:
        if isSupport(p) and p["teamId"]==teamId:
            return True
    return False

In [4]:
#Analysis of the importance of leading the game
#A leading team is a team having at least 6 kills more than its opponent (value arbitrarily set)

#Check if one team has a lead on the other, and which one then
def getLeadingTeam(kills100, kills200):
    if kills100 >= kills200 + 6:
        return 100
    elif kills200 >= kills100 + 6:
        return 200
    return None


def getFirstLeadingTeam(game):
    kills100 = 0
    kills200 = 0
    for frame in game['timeline']['frames']:
        for event in frame['events']:
            if event['type'] == "CHAMPION_KILL":
                if event["killerId"] in [1,2,3]:
                    kills100+=1
                else:
                    kills200+=1
                team = getLeadingTeam(kills100, kills200)
                if not team == None:
                    return team

In [5]:
#Some personal stats
def hasAnivia(game, teamId):
    for p in game['participants']:
        if p["championId"]==34 and p["teamId"]==teamId:
            return True
    return False

def hasVelkoz(game, teamId):
    for p in game['participants']:
        if p["championId"]==161 and p["teamId"]==teamId:
            return True
    return False

In [6]:
#Analysis for games since patch 8.15
startTimestamp = 1533097800 * 1000

In [7]:
gamesData = []
for game in mongoTable.find( { "gameCreation": { "$gt": startTimestamp } }):
    if game["gameDuration"] < 300:
        continue
    
    winningTeam = getWinner(game)
    loosingTeam = 200 if winningTeam==100 else 100
    
    winnerMeta = "hypercarry" if hasSupport(game,winningTeam) else "jungler"
    looserMeta = "hypercarry" if hasSupport(game,loosingTeam) else "jungler"
    
    leadingTeam = getFirstLeadingTeam(game)
    
    winningTeamHasAnivia = hasAnivia(game,winningTeam)
    loosingTeamHasAnivia = hasAnivia(game,loosingTeam)
    
    winningTeamHasVelkoz = hasVelkoz(game,winningTeam)
    loosingTeamHasVelkoz = hasVelkoz(game,loosingTeam)
    
    gamesData.append({
        "winningTeam":winningTeam,
        "loosingTeam":loosingTeam,
        "winnerMeta":winnerMeta,
        "looserMeta":looserMeta,
        "leadingTeam":leadingTeam,
        "winningTeamHasAnivia":winningTeamHasAnivia,
        "loosingTeamHasAnivia":loosingTeamHasAnivia,
        "winningTeamHasVelkoz":winningTeamHasVelkoz,
        "loosingTeamHasVelkoz":loosingTeamHasVelkoz
    })

In [8]:
import pandas as pd
df = pd.DataFrame(gamesData)

In [9]:
#Winrate hypercarry VS jungler
#Games hypercarry winning on jungler
dfHcWinning = df[((df["winnerMeta"]=="hypercarry") & (df["looserMeta"]=="jungler"))]
#Games jungler winning on hypercarry
dfJunWinning = df[((df["winnerMeta"]=="jungler") & (df["looserMeta"]=="hypercarry"))]

In [10]:
#Hypercarry winrate against Jungler
hcWinrate = dfHcWinning.shape[0]/(dfHcWinning.shape[0]+dfJunWinning.shape[0])

In [11]:
#Versus count
gameCount = {
    "jglVSjgl":df[((df["winnerMeta"]=="jungler") & (df["looserMeta"]=="jungler"))].shape[0],
    "hcVShc":df[((df["winnerMeta"]=="hypercarry") & (df["looserMeta"]=="hypercarry"))].shape[0],
    "hcVSjgl":pd.concat([dfHcWinning,dfJunWinning]).shape[0],
    "total":df.shape[0]
}

In [12]:
#number of games with a leading team
nbGameWithLead = df[((df["leadingTeam"]==100) | (df["leadingTeam"]==200))].shape[0]

In [13]:
#winrate of leading team actually winning the game
nbLeadingTeamWinning = df[df["leadingTeam"]==df["winningTeam"]].shape[0]
winrateLeadingTeam = nbLeadingTeamWinning/df[((df["leadingTeam"]==100) | (df["leadingTeam"]==200))].shape[0]
#percentage of game with a leading team
percentageLeadingTeam = nbGameWithLead/df.shape[0]

In [14]:
#winrate of leading team actually winning the game in jungler VS jungler
nbLeadingTeamWinningJungle = df[((df["leadingTeam"]==df["winningTeam"]) & (df["winnerMeta"]=="jungler") & (df["looserMeta"]=="jungler"))].shape[0]
nbGameWithLeadJungle = df[(((df["leadingTeam"]==100) | (df["leadingTeam"]==200)) & (df["winnerMeta"]=="jungler") & (df["looserMeta"]=="jungler"))].shape[0]

winrateLeadingTeamJungle = nbLeadingTeamWinningJungle/nbGameWithLeadJungle
#percentage of game with a leading team in jungler VS jungler
percentageLeadingTeamJungle = nbGameWithLeadJungle/gameCount["jglVSjgl"]

In [15]:
#winrate of leading team actually winning the game in hypercarry VS hypercarry
nbLeadingTeamWinningHypercarry = df[((df["leadingTeam"]==df["winningTeam"]) & (df["winnerMeta"]=="hypercarry") & (df["looserMeta"]=="hypercarry"))].shape[0]
nbGameWithLeadHypercarry = df[(((df["leadingTeam"]==100) | (df["leadingTeam"]==200)) & (df["winnerMeta"]=="hypercarry") & (df["looserMeta"]=="hypercarry"))].shape[0]

winrateLeadingTeamHypercarry = nbLeadingTeamWinningHypercarry/nbGameWithLeadHypercarry
#percentage of game with a leading team in hypercarry VS hypercarry
percentageLeadingTeamHypercarry = nbGameWithLeadHypercarry/gameCount["hcVShc"]

In [16]:
#winrate of leading team actually winning the game in hypercarry VS jungler
dfBoth = pd.concat([dfHcWinning,dfJunWinning])
nbLeadingTeamWinningBoth = dfBoth[dfBoth["leadingTeam"]==dfBoth["winningTeam"]].shape[0]
nbGameWithLeadBoth = dfBoth[((dfBoth["leadingTeam"]==100) | (dfBoth["leadingTeam"]==200))].shape[0]

winrateLeadingTeamBoth = nbLeadingTeamWinningBoth/nbGameWithLeadBoth
#percentage of game with a leading team in hypercarry VS jungler
percentageLeadingTeamBoth = nbGameWithLeadBoth/gameCount["hcVSjgl"]

In [17]:
#Red team winrate
redTeamWinrate = df[df["winningTeam"]==200].shape[0] / df.shape[0]
redTeamWinrateJungle = df[((df["winningTeam"]==200) & (df["winnerMeta"]=="jungler") & (df["looserMeta"]=="jungler"))].shape[0] /gameCount["jglVSjgl"]
redTeamWinrateHypercarry = df[((df["winningTeam"]==200) & (df["winnerMeta"]=="hypercarry") & (df["looserMeta"]=="hypercarry"))].shape[0] /gameCount["hcVShc"]
redTeamWinrateBoth = dfBoth[dfBoth["winningTeam"]==200].shape[0] /gameCount["hcVSjgl"]

In [18]:
stats = []

stats.append({
    "Meta":"All",
    "nbGames":gameCount["total"],
    "percentageTotalGame":gameCount["total"]/gameCount["total"],
    "nbGameWithLead":nbGameWithLead,
    "percentageLeadingTeam":percentageLeadingTeam,
    "winrateLeadingTeam":winrateLeadingTeam,
    "redTeamWinrate":redTeamWinrate
})

stats.append({
    "Meta":"Jungle Only",
    "nbGames":gameCount["jglVSjgl"],
    "percentageTotalGame":gameCount["jglVSjgl"]/gameCount["total"],
    "nbGameWithLead":nbGameWithLeadJungle,
    "percentageLeadingTeam":percentageLeadingTeamJungle,
    "winrateLeadingTeam":winrateLeadingTeamJungle,
    "redTeamWinrate":redTeamWinrateJungle
})

stats.append({
    "Meta":"Hypercarry only",
    "nbGames":gameCount["hcVShc"],
    "percentageTotalGame":gameCount["hcVShc"]/gameCount["total"],
    "nbGameWithLead":nbGameWithLeadHypercarry,
    "percentageLeadingTeam":percentageLeadingTeamHypercarry,
    "winrateLeadingTeam":winrateLeadingTeamHypercarry,
    "redTeamWinrate":redTeamWinrateHypercarry
})

stats.append({
    "Meta":"Hypercarry VS Jungle",
    "nbGames":gameCount["hcVSjgl"],
    "percentageTotalGame":gameCount["hcVSjgl"]/gameCount["total"],
    "nbGameWithLead":nbGameWithLeadBoth,
    "percentageLeadingTeam":percentageLeadingTeamBoth,
    "winrateLeadingTeam":winrateLeadingTeamBoth,
    "redTeamWinrate":redTeamWinrateBoth
})

In [19]:
dfStats = pd.DataFrame(stats)
dfStats["percentageTotalGame"] = (dfStats["percentageTotalGame"]*100).map('{:,.2f}'.format)
dfStats["percentageLeadingTeam"] = (dfStats["percentageLeadingTeam"]*100).map('{:,.2f}'.format)
dfStats["winrateLeadingTeam"] = (dfStats["winrateLeadingTeam"]*100).map('{:,.2f}'.format)
dfStats["redTeamWinrate"] = (dfStats["redTeamWinrate"]*100).map('{:,.2f}'.format)

In [20]:
#Anivia
aniviaPickrate = df[((df["winningTeamHasAnivia"]) | (df["loosingTeamHasAnivia"]))].shape[0] / gameCount["total"]
aniviaWinrate = df[df["winningTeamHasAnivia"]].shape[0]/df[((df["winningTeamHasAnivia"]) | (df["loosingTeamHasAnivia"]))].shape[0]

In [21]:
#Velkoz
velkozPickrate = df[((df["winningTeamHasVelkoz"]) | (df["loosingTeamHasVelkoz"]))].shape[0] / gameCount["total"]
velkozWinrate = df[df["winningTeamHasVelkoz"]].shape[0]/df[((df["winningTeamHasVelkoz"]) | (df["loosingTeamHasVelkoz"]))].shape[0]

In [22]:
championStats = []
championStats.append({
    "champion":"Anivia",
    "pickrate":aniviaPickrate,
    "winrate":aniviaWinrate
})
championStats.append({
    "champion":"Velkoz",
    "pickrate":velkozPickrate,
    "winrate":velkozWinrate
})
dfChampionStats = pd.DataFrame(championStats)
dfChampionStats["pickrate"] = (dfChampionStats["pickrate"]*100).map('{:,.2f}'.format)
dfChampionStats["winrate"] = (dfChampionStats["winrate"]*100).map('{:,.2f}'.format)

# Results

Winrate Hypercarry VS Jungle

In [23]:
"{:.2f}%".format(hcWinrate*100)

'64.58%'

Stats on game sorted by meta
 - nbGames : Total number of games
 - percentageTotalGame : Percentage of game following the specified meta
 - nbGameWithLead : Number of games where a team has a lead of at least 6 kills
 - percentageLeadingTeam : Percentage of games with a leading team on the total
 - winrateLeadingTeam : Winrate of the leading team
 - redTeamWinrate : Winrate of the Red team

In [24]:
dfStats[["Meta","nbGames","percentageTotalGame","nbGameWithLead","percentageLeadingTeam","winrateLeadingTeam","redTeamWinrate"]]

,Meta,nbGames,percentageTotalGame,nbGameWithLead,percentageLeadingTeam,winrateLeadingTeam,redTeamWinrate
0,All,2293,100.00,1854,80.85,93.53,59.01
1,Jungle Only,321,14.00,272,84.74,92.65,56.07
2,Hypercarry only,1156,50.41,928,80.28,95.15,59.43
3,Hypercarry VS Jungle,816,35.59,654,80.15,91.59,59.56


Pickrate and Winrate for Anivia and Velkoz

In [25]:
dfChampionStats

,champion,pickrate,winrate
0,Anivia,11.43,45.04
1,Velkoz,22.98,59.01


In [27]:
dfStats.to_csv("statsEUWafter.csv", index=False)